In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl as opx
import numpy as np
import folium 

In [3]:
sementeiras_rs = gpd.read_file('C:\Git\personal-two\consulta\Sementeiras\dados\sementeiras_rs.json')
municipios_rs = gpd.read_file('C:\Git\personal-two\consulta\Sementeiras\dados\municipios_rs.json')
centroides = gpd.read_file('C:\Git\personal-two\consulta\Sementeiras\dados\centroides.json.geojson')

In [3]:
sementeiras_rs

,Safra,Especie,Categoria,Cultivar,Municipio,UF,Status,Data do Plantio,Data de Colheita,Area,Producao bruta,Producao estimada,geometry
0,2013/2013,Glycine max (L.) Merr.,S2,BMX Força RR,Chapada,RS,Homologado,11/02/2013,,82.0,,328.0,None
1,2013/2013,Raphanus sativus L.,S2,Comum,Santa Bárbara do Sul,RS,Homologado,03/04/2013,,170.0,,204.0,None
2,2013/2013,Vicia sativa L.,S2,ESPÉCIE,Três de Maio,RS,Homologado,23/05/2013,,12.0,,18.0,None
3,2013/2013,Avena strigosa Schreb.,S2,Embrapa 139,Três de Maio,RS,Homologado,23/05/2013,,20.0,,40.0,None
4,2013/2013,Avena sativa L. = Avena byzantina K. Koch,S2,FUNDACEPFAPA 43,Independência,RS,Homologado,21/05/2013,,30.0,,60.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75689,2022/2023,Glycine max (L.) Merr.,S1,NS5933IPRO,Dilermando de Aguiar,RS,Inscrito,04/11/2022,,120.0,,201.48,None
75690,2022/2023,Glycine max (L.) Merr.,S1,64I61RSF IPRO,Palmeira das Missões,RS,Inscrito,31/10/2022,,80.0,,312.0,None
75691,2022/2023,Glycine max (L.) Merr.,C2,59I60RSF IPRO,Palmeira das Missões,RS,Inscrito,30/10/2022,,50.0,,200.0,None
75692,2022/2023,Glycine max (L.) Merr.,S2,BMX ATIVA RR,Não-Me-Toque,RS,Inscrito,09/12/2022,,66.0,,264.0,None


In [4]:
sementeiras_rs.drop(["Categoria", "UF", "Status", "Data de Colheita", 
                     "Data de Colheita", "Producao bruta", "geometry"], axis=1, inplace=True)
municipios_rs.drop(["SIGLA"], axis=1, inplace=True)
municipiosm_rs = municipios_rs.rename(columns={'NM_MUN': 'Municipio'})
municipios = municipiosm_rs.sort_values('Municipio')
uniao = municipiosm_rs.merge(sementeiras_rs, on='Municipio')


In [5]:
municipios.head(0)

,CD_MUN,Municipio,AREA_KM2,geometry


In [6]:
sementeiras_rs

,Safra,Especie,Cultivar,Municipio,Data do Plantio,Area,Producao estimada
0,2013/2013,Glycine max (L.) Merr.,BMX Força RR,Chapada,11/02/2013,82.0,328.0
1,2013/2013,Raphanus sativus L.,Comum,Santa Bárbara do Sul,03/04/2013,170.0,204.0
2,2013/2013,Vicia sativa L.,ESPÉCIE,Três de Maio,23/05/2013,12.0,18.0
3,2013/2013,Avena strigosa Schreb.,Embrapa 139,Três de Maio,23/05/2013,20.0,40.0
4,2013/2013,Avena sativa L. = Avena byzantina K. Koch,FUNDACEPFAPA 43,Independência,21/05/2013,30.0,60.0
...,...,...,...,...,...,...,...
75689,2022/2023,Glycine max (L.) Merr.,NS5933IPRO,Dilermando de Aguiar,04/11/2022,120.0,201.48
75690,2022/2023,Glycine max (L.) Merr.,64I61RSF IPRO,Palmeira das Missões,31/10/2022,80.0,312.0
75691,2022/2023,Glycine max (L.) Merr.,59I60RSF IPRO,Palmeira das Missões,30/10/2022,50.0,200.0
75692,2022/2023,Glycine max (L.) Merr.,BMX ATIVA RR,Não-Me-Toque,09/12/2022,66.0,264.0


In [8]:
soja = uniao[uniao['Especie'] == 'Glycine max (L.) Merr.']
soja22 = soja[soja['Safra'] == '2021/2022']
soja21 = soja[soja['Safra'] == '2021/2021']
soja22 = pd.concat([soja22, soja21])
# soja22 = soja21.append(soja22)
soja22 = soja22.sort_values('Municipio')
soja22['Area'] = soja22['Area'].astype('float')
soja22['Producao estimada'] = soja22['Producao estimada'].astype('float')
soja22 = soja22.dropna(subset=['Area', 'Producao estimada'])
soja22.dtypes

CD_MUN                 object
Municipio              object
AREA_KM2              float64
geometry             geometry
Safra                  object
Especie                object
Cultivar               object
Data do Plantio        object
Area                  float64
Producao estimada     float64
dtype: object

In [9]:
soja22

,CD_MUN,Municipio,AREA_KM2,geometry,Safra,Especie,Cultivar,Data do Plantio,Area,Producao estimada
144,4300034,Aceguá,1551.339,"POLYGON ((-54.10842 -31.43275, -54.10739 -31.4...",2021/2022,Glycine max (L.) Merr.,NS6700IPRO,22/10/2021,22.0,92.4
154,4300034,Aceguá,1551.339,"POLYGON ((-54.10842 -31.43275, -54.10739 -31.4...",2021/2022,Glycine max (L.) Merr.,66I68RSF IPRO,01/12/2021,200.0,780.0
153,4300034,Aceguá,1551.339,"POLYGON ((-54.10842 -31.43275, -54.10739 -31.4...",2021/2022,Glycine max (L.) Merr.,6401XTD,14/11/2021,40.0,144.0
151,4300034,Aceguá,1551.339,"POLYGON ((-54.10842 -31.43275, -54.10739 -31.4...",2021/2022,Glycine max (L.) Merr.,AS 3590IPRO,21/11/2021,145.0,522.0
150,4300034,Aceguá,1551.339,"POLYGON ((-54.10842 -31.43275, -54.10739 -31.4...",2021/2022,Glycine max (L.) Merr.,AS 3590IPRO,14/11/2021,235.0,846.0
...,...,...,...,...,...,...,...,...,...,...
75368,4323457,Vila Nova do Sul,508.278,"POLYGON ((-53.95465 -30.20750, -53.95408 -30.2...",2021/2022,Glycine max (L.) Merr.,6968 RSF,20/11/2021,250.0,750.0
75373,4323457,Vila Nova do Sul,508.278,"POLYGON ((-53.95465 -30.20750, -53.95408 -30.2...",2021/2022,Glycine max (L.) Merr.,64I61RSF IPRO,01/11/2021,74.0,310.0
294,4300059,Água Santa,291.503,"POLYGON ((-52.04086 -28.12188, -52.04031 -28.1...",2021/2022,Glycine max (L.) Merr.,BMX ATIVA RR,02/12/2021,66.0,317.0
296,4300059,Água Santa,291.503,"POLYGON ((-52.04086 -28.12188, -52.04031 -28.1...",2021/2022,Glycine max (L.) Merr.,BMX ATIVA RR,30/11/2021,27.0,129.0


In [10]:
areas = list(soja22.groupby('Municipio').sum()['Area'])
soja22 = soja22.drop_duplicates(subset='Municipio')
soja22 = soja22.assign(Area=areas)

C:\Users\isach\AppData\Local\Temp\ipykernel_13872\246644608.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  areas = list(soja22.groupby('Municipio').sum()['Area'])


In [11]:
soja22.head(1)


,CD_MUN,Municipio,AREA_KM2,geometry,Safra,Especie,Cultivar,Data do Plantio,Area,Producao estimada
144,4300034,Aceguá,1551.339,"POLYGON ((-54.10842 -31.43275, -54.10739 -31.4...",2021/2022,Glycine max (L.) Merr.,NS6700IPRO,22/10/2021,1233.0,92.4


In [16]:
latitude = -29.94
logitude = -53.10
fmap = folium.Map(location=[latitude, logitude], tiles='cartodbpositron', zoom_start= 7)
limites = folium.features.GeoJson(municipios_rs, 
                                  style_function= lambda feature: {
                                      'color': 'black',
                                      'weght':1,
                                      'fillOpacity': 0.0
                                  })
fmap.add_child(limites)

#for _, Municipio in centroides.iterrows():
lsoja = folium.features.GeoJson(centroides,
                                style_function= lambda feature: {
                                      'color': 'red',
                                      'weght':2,
                                      'height':3,
                                      'fillOpacity': 0.5
                                  })
fmap.add_child(lsoja)
fmap

IndentationError: expected an indented block after 'for' statement on line 12 (3831816408.py, line 13)

In [15]:
latitude = -29.94
logitude = -53.10
fmap = folium.Map(location=[latitude, logitude], tiles='cartodbpositron', zoom_start= 7)
for _, municipio in municipios.iterrows():
  mprodutores = folium.features.GeoJson(municipio.geometry, 
                                    style_function= lambda feature: {
                                        'color': 'black',
                                        'weght':1,
                                        'fillOpacity': 0.0
                                    })
  fmap.add_child(mprodutores)
for _, municipio in soja22.iterrows():
  limites = folium.features.GeoJson(municipio.geometry, 
                                    style_function= lambda feature: {
                                        'color': 'blue',
                                        'weght':2,
                                        'fillOpacity': 0.1
                                    })
  popup = folium.Popup(f"""<p align=”center”><b>Área de produção de sementes de soja no RS</b></center><br>
                          Município: {municipio.Municipio} <br>
                          Área estimada (2021/2022): {str(int(municipio.Area))}ha  
                          """, max_width=300)
  popup.add_to(limites)

  limites.add_to(fmap)

fmap.save('C:/Git/pessoal-one/consulta/Sementeiras/dados/area_produzida_2022.html')

